In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from keras.utils import Sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle

Using TensorFlow backend.


#### If using AMD GPU:

In [ ]:
import os
os.environ['KERAS_BACKEND']='plaidml.keras.backend'

# When using plaidml, the libraries are imported from keras instead of tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

#### Set up tensorboard

In [3]:
from tensorflow.keras.callbacks import TensorBoard
import time
NAME = 'Age-prediction-{}'.format(int(time.time()))  # model name with timestamp
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

#### Load input data labels

In [6]:
pickle_in = open('../Dataset/labels.pickle', 'rb')
df = pickle.load(pickle_in)

In [7]:
df

path     id           name  \
0         01/nm0000001_rm124825600_1899-5-10_1968.jpg   6488   Fred Astaire   
1        01/nm0000001_rm3343756032_1899-5-10_1970.jpg   6488   Fred Astaire   
2         01/nm0000001_rm577153792_1899-5-10_1968.jpg   6488   Fred Astaire   
3         01/nm0000001_rm946909184_1899-5-10_1968.jpg   6488   Fred Astaire   
4         01/nm0000001_rm980463616_1899-5-10_1968.jpg   6488   Fred Astaire   
5        02/nm0000002_rm1075631616_1924-9-16_1991.jpg  11516  Lauren Bacall   
6        02/nm0000002_rm1346607872_1924-9-16_2004.jpg  11516  Lauren Bacall   
7        02/nm0000002_rm1363385088_1924-9-16_2004.jpg  11516  Lauren Bacall   
8        02/nm0000002_rm1411175936_1924-9-16_1991.jpg  11516  Lauren Bacall   
9        02/nm0000002_rm1447271168_1924-9-16_2004.jpg  11516  Lauren Bacall   
10       02/nm0000002_rm1624085760_1924-9-16_2008.jpg  11516  Lauren Bacall   
11       02/nm0000002_rm1646056960_1924-9-16_1991.jpg  11516  Lauren Bacall   
12        02/nm0000002_rm221957120_1924-9-16_1974.jpg  11516  Lauren Bacall   
13       02/nm0000002_rm2287049216_1924-9-16_2007.jpg  11516  Lauren Bacall   
14        02/nm0000002_rm238734336_1924-9-16_1974.jpg  11516  Lauren Bacall   
15       02/nm0000002_rm2585828096_1924-9-16_2006.jpg  11516  Lauren Bacall   
16       02/nm0000002_rm2602605312_1924-9-16_2006.jpg  11516  Lauren Bacall   
17       02/nm0000002_rm2619382528_1924-9-16_2006.jpg  11516  Lauren Bacall   
18       02/nm0000002_rm2769394176_1924-9-16_2006.jpg  11516  Lauren Bacall   
19       02/nm0000002_rm2780403712_1924-9-16_2004.jpg  11516  Lauren Bacall   
20       02/nm0000002_rm2805435904_1924-9-16_1988.jpg  11516  Lauren Bacall   
21       02/nm0000002_rm2813958144_1924-9-16_2004.jpg  11516  Lauren Bacall   
22        02/nm0000002_rm289065984_1924-9-16_1974.jpg  11516  Lauren Bacall   
23       02/nm0000002_rm2983566080_1924-9-16_2007.jpg  11516  Lauren Bacall   
24        02/nm0000002_rm305843200_1924-9-16_1974.jpg  11516  Lauren Bacall   
25       02/nm0000002_rm3224279040_1924-9-16_2005.jpg  11516  Lauren Bacall   
26       02/nm0000002_rm3453524224_1924-9-16_2007.jpg  11516  Lauren Bacall   
27       02/nm0000002_rm3487078656_1924-9-16_2007.jpg  11516  Lauren Bacall   
28        02/nm0000002_rm350394368_1924-9-16_2005.jpg  11516  Lauren Bacall   
29       02/nm0000002_rm3537671424_1924-9-16_2008.jpg  11516  Lauren Bacall   
...                                               ...    ...            ...   
460693   08/nm3994408_rm350134272_1989-12-29_2013.jpg   8410      Jane Levy   
460694  08/nm3994408_rm3532828160_1989-12-29_2012.jpg   8410      Jane Levy   
460695   08/nm3994408_rm357150464_1989-12-29_2011.jpg   8410      Jane Levy   
460696  08/nm3994408_rm3650268672_1989-12-29_2012.jpg   8410      Jane Levy   
460697  08/nm3994408_rm3713712128_1989-12-29_2014.jpg   8410      Jane Levy   
460698     08/nm3994408_rm3845632_1989-12-29_2011.jpg   8410      Jane Levy   
460699  08/nm3994408_rm3926507520_1989-12-29_2014.jpg   8410      Jane Levy   
460700  08/nm3994408_rm4062552064_1989-12-29_2011.jpg   8410      Jane Levy   
460701  08/nm3994408_rm4163215360_1989-12-29_2011.jpg   8410      Jane Levy   
460702  08/nm3994408_rm4179992576_1989-12-29_2011.jpg   8410      Jane Levy   
460703   08/nm3994408_rm474590976_1989-12-29_2011.jpg   8410      Jane Levy   
460704   08/nm3994408_rm524922624_1989-12-29_2011.jpg   8410      Jane Levy   
460705   08/nm3994408_rm540716544_1989-12-29_2011.jpg   8410      Jane Levy   
460706     08/nm3994408_rm5419776_1989-12-29_2011.jpg   8410      Jane Levy   
460707   08/nm3994408_rm543141888_1989-12-29_2011.jpg   8410      Jane Levy   
460708   08/nm3994408_rm559919104_1989-12-29_2011.jpg   8410      Jane Levy   
460709    08/nm3994408_rm56609536_1989-12-29_2011.jpg   8410      Jane Levy   
460710   08/nm3994408_rm591048192_1989-12-29_2011.jpg   8410      Jane Levy   
460711   08/nm3994408_rm605006080_1989-12-29_2012.jpg   8410      Jane Levy   
460712   08/nm399440

In [ ]:
image_reshape_size = 100
data_root = ''

inputShape = 
training_filenames = 
validation_filenames = 
GT_training = 
GT_validation = 
batch_size = 

#### Set up input data generator

In [12]:
class DataGenerator(Sequence):

    def __init__(self, image_filenames, labels, batch_size):
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size

    def __len__(self):
        return np.ceil(len(self.image_filenames) / float(self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        return np.array([resize(imread(file_name), (image_reshape_size, image_reshape_size)) 
                         for file_name in batch_x]), np.array(batch_y)

#### Set up model

In [13]:
model = Sequential()

# first layer (Conv2D)
model.add(Conv2D(32, (3,3), input_shape=inputShape)) #32 neurons, 3X3 window, shape50x50x1
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# second layer (Conv2D)
model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# third layer (Conv2D)
model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Fourth layer (Dense)
model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))

# output layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam',
             metrics=['accuracy'])

train_generator = DataGenerator(training_filenames, GT_training, batch_size)
test_generator = DataGenerator(validation_filenames, GT_validation, batch_size)

NameError: name 'X' is not defined

In [ ]:
# model.fit(X, y, batch_size=32, epochs=10, validation_split=0.3, callbacks=[tensorboard]) 
# batch_size: how many samples to pass each time, scale it relative to the size of total training set

model.fit_generator(generator=my_training_batch_generator,
                    steps_per_epoch=(num_training_samples // batch_size),
                    epochs=10
                    verbose=1,
                    validation_data=train_generator,
                    validation_steps=(num_validation_samples // batch_size),
                    use_multiprocessing=True,
                    workers=16,
                    max_queue_size=32)